# Project: Investigate a Dataset - [TMDB Movie Dataset]

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description

This project centers on a cleaned version of the **TMDb movie dataset** originally sourced from [Kaggle](https://www.kaggle.com/tmdb/tmdb-movie-metadata). The dataset includes information on approximately 10,000 movies such as:

- **title**: The official title of the movie.
- **id**: A unique identifier for each movie.
- **genres**: A pipe-separated list of genres associated with the film.
- **cast**: A pipe-separated list of main cast members.
- **director**: The name of the primary director for the film.
- **budget_adj** and **revenue_adj**: The budget and revenue in terms of 2010 dollars, adjusted for inflation.
- **popularity**: A TMDb-defined metric reflecting user activity and interest.
- **vote_average**: The average user rating on TMDb, which will be a key focus in our analysis.
- **vote_count**: The total number of user votes.
- **release_date**: The film’s release date.

For this project, we will look specifically at **director–actor collaborations** to explore how certain factors may correlate with audience reception. After some data wrangling, we plan to derive new variables such as:

- **`collaboration_count`**: How many times a given director–actor pair has worked together.
- **`dominant_genre`**: The most frequently appearing genre for each director–actor pair.
- **`avg_vote_average`**: The average rating (on a scale of 0–10) for all films created by that pair.

Our goal is to investigate relationships between these variables (and potentially others) to gain insight into which director–actor teams may achieve higher audience ratings and whether certain genres appear more strongly correlated with positive viewer feedback. This exploration will be primarily **descriptive** and **exploratory**—we are looking for interesting patterns in the data rather than drawing definitive causal conclusions.



### Question(s) for Analysis

Our main goal in this project is to explore **how certain characteristics of director–actor pairs relate to audience reception** (as measured by **`avg_vote_average`**). To guide our investigation, we will focus on the following questions:

1. **Does the number of collaborations between a director and an actor (`collaboration_count`) correspond to higher or lower average audience ratings (`avg_vote_average`)?**  
2. **Does the primary genre in which a director–actor pair collaborates (`dominant_genre`) appear to influence their films’ average ratings?**  
3. **Is there a relationship between additional factors (e.g., `popularity`, `avg_revenue_adj`, or release years) and audience ratings for these pairs?**

We chose to use **`avg_vote_average`** as our **dependent variable**, given that it directly measures audience reception. Our **independent variables** of interest will be:

- **`collaboration_count`** (numeric)  
- **`dominant_genre`** (categorical)  
- **A third variable** (e.g., `popularity`, `avg_revenue_adj`, or another feature of interest)

<a id='wrangling'></a>
## Data Wrangling

In this section, we will **load**, **inspect**, and **clean** our TMDb dataset. Our aim is to transform the data so that it is suitable for analyzing the relationships between director–actor pairs and their average audience ratings.

### General Properties

Below are the steps we will follow:

1. **Load the dataset** from a CSV file into a Pandas DataFrame.
2. **Explore the DataFrame**: Check dimensions, column datatypes, and sample rows.
3. **Assess data quality**: Look for missing values, invalid entries, or duplicates.
4. **Clean data**: Drop unnecessary columns, handle missing or zero values if needed, and create or transform columns that will help our analysis (e.g., `collaboration_count`, `dominant_genre`).



### General Properties


In [ ]:
import pandas as pd
import numpy as np

# Load the TMDb dataset
df = pd.read_csv('tmdb-movies.csv')

# Quick preview of the data
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'tmdb_movies.csv'


### Data Cleaning
> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).
 

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. **Compute statistics** and **create visualizations** with the goal of addressing the research questions that you posed in the Introduction section. You should compute the relevant statistics throughout the analysis when an inference is made about the data. Note that at least two or more kinds of plots should be created as part of the exploration, and you must  compare and show trends in the varied visualizations. Remember to utilize the visualizations that the pandas library already has available.



> **Tip**: Investigate the stated question(s) from multiple angles. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables. You should explore at least three variables in relation to the primary question. This can be an exploratory relationship between three variables of interest, or looking at how two independent variables relate to a single dependent variable of interest. Lastly, you  should perform both single-variable (1d) and multiple-variable (2d) explorations.

### Research Question 1 (Replace this header name!)

### Research Question 2  (Replace this header name!)

<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed in relation to the question(s) provided at the beginning of the analysis. Summarize the results accurately, and point out where additional research can be done or where additional information could be useful.

> **Tip**: Make sure that you are clear with regards to the limitations of your exploration. You should have at least 1 limitation explained clearly. 

> **Tip**: If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.

## Submitting your Project 

> **Tip**: Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should see output that starts with `NbConvertApp] Converting notebook`, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> **Tip**: Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> **Tip**: Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!